# Dependencies

For ZnTrack there are two different ways to set up dependencies:

1. Node/Stage based dependencies
2. File based dependencies

## Node dependencies
We will first look at Node based dependencies starting from a RandomNumber `Hello World` example.
In our first stage we create a random number and then we add another Node that depends on this one.
We can do this very easily by setting the `dvc.deps` in the following way
```py
dependency: Stage = dvc.deps(Stage(load=True))
```

This allows us to access all properties of the `dependency` attribute.

In [1]:
from zntrack import Node, dvc, zn, config
from random import randrange
from pathlib import Path

In [2]:
config.nb_name = "03_dependencies.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir
temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmp786zcf_9/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [5]:
@Node()
class RandomNumber:
    maximum = dvc.params()
    number = zn.outs()
    
    def __call__(self, maximum):
        self.maximum = maximum
        
    def run(self):
        self.number = float(randrange(self.maximum))

@Node()
class ComputePower:
    random_number: RandomNumber = dvc.deps(RandomNumber(load=True))
    number = dvc.result()
    power = dvc.params()
    
    def __call__(self, power = 2):
        self.power = power
    
    def run(self):
        self.number = self.random_number.number ** self.power

2021-11-30 16:17:50,049 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


2021-11-30 16:17:53,411 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3414 bytes to 03_dependencies.py
[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
[NbConvertApp] Writing 3414 bytes to 03_dependencies.py


We can now create the stages the usual way and look at the outcomes

In [6]:
RandomNumber()(maximum=16)
ComputePower()(power=2.)

2021-11-30 16:17:57,086 (WARNING): --- Writing new DVC file! ---
2021-11-30 16:17:59,833 (INFO): Creating 'dvc.yaml'
Adding stage 'RandomNumber' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml nodes/RandomNumber/.gitignore

2021-11-30 16:17:59,837 (WARNING): --- Writing new DVC file! ---
2021-11-30 16:18:02,388 (INFO): Adding stage 'ComputePower' in 'dvc.yaml'

To track the changes with git, run:

	git add nodes/ComputePower/.gitignore dvc.yaml



In [7]:
!dvc repro

Running stage 'RandomNumber':                                         core>
> python3 -c "from src.RandomNumber import RandomNumber; RandomNumber(load=True, name='RandomNumber').run()" 
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

Running stage 'ComputePower':
> python3 -c "from src.ComputePower import ComputePower; ComputePower(load=True, name='ComputePower').run()" 
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [8]:
print(f'{RandomNumber(load=True).number} ^ {ComputePower(load=True).power} = {ComputePower(load=True).number}')

2021-11-30 16:18:06,577 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


2021-11-30 16:18:09,824 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3414 bytes to 03_dependencies.py
[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


2021-11-30 16:18:13,009 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


...

[NbConvertApp] Writing 3414 bytes to 03_dependencies.py
[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


2021-11-30 16:18:19,625 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3414 bytes to 03_dependencies.py
[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


0.0 ^ 2.0 = 0.0


[NbConvertApp] Writing 3414 bytes to 03_dependencies.py


## File dependencies
The second approach is depending on files.
We do this by creating a file from our random number first.
We then use the path to that file as our dependency.
In the end we compare both methods and also show the graph that was built for us by DVC.

We can do set the file dependency very easily in the same way by passing `pathlib.Path` or `str` to the `dvc.deps`.
As every  `dvc.<...>` it also supports lists.
```py
dependency: Path = dvc.deps([Path('some_file.txt'), 'some_other_file.txt'])
```

In [9]:
@Node()
class WriteToFile:
    random_number: RandomNumber = dvc.deps(RandomNumber(load=True))
    file: Path = dvc.outs(Path('random_number.txt'))
    
    def run(self):
        self.file.write_text(str(self.random_number.number))
        
@Node()
class PowerFromFile:
    file: Path = dvc.deps(WriteToFile(load=True).file)
    number = zn.outs()
    power = dvc.params(2)
    
    def run(self):
        number = float(self.file.read_text())
        self.number = number ** self.power
        
@Node()
class ComparePowers:
    power_deps = dvc.deps([PowerFromFile(load=True), ComputePower(load=True)])
    
    def run(self):
        assert self.power_deps[0].number == self.power_deps[1].number

2021-11-30 16:18:22,818 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


2021-11-30 16:18:25,973 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3414 bytes to 03_dependencies.py
[NbConvertApp] Converting notebook 03_dependencies.ipynb to script


2021-11-30 16:18:29,118 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3414 bytes to 03_dependencies.py
[NbConvertApp] Converting notebook 03_dependencies.ipynb to script
[NbConvertApp] Writing 3414 bytes to 03_dependencies.py


In [10]:
WriteToFile()()
PowerFromFile()()
ComparePowers()()

2021-11-30 16:18:32,334 (WARNING): --- Writing new DVC file! ---
2021-11-30 16:18:34,959 (INFO): Adding stage 'WriteToFile' in 'dvc.yaml'

To track the changes with git, run:

	git add .gitignore dvc.yaml

2021-11-30 16:18:34,964 (WARNING): --- Writing new DVC file! ---
2021-11-30 16:18:37,493 (INFO): Adding stage 'PowerFromFile' in 'dvc.yaml'

To track the changes with git, run:

	git add nodes/PowerFromFile/.gitignore dvc.yaml

2021-11-30 16:18:37,496 (WARNING): --- Writing new DVC file! ---
2021-11-30 16:18:40,009 (INFO): Adding stage 'ComparePowers' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml



In [11]:
!dvc dag

                +--------------+                  
                | RandomNumber |                  
                +--------------+                  
                **             ***                
             ***                  ***             
           **                        **           
 +-------------+                       **         
 | WriteToFile |                        *         
 +-------------+                        *         
        *                               *         
        *                               *         
        *                               *         
+---------------+               +--------------+  
| PowerFromFile |               | ComputePower |  
+---------------+               +--------------+  
                **             ***                
                  ***        **                   
                     **    **                     
                +---------------+                 
                | ComparePowers

In [12]:
!dvc repro

Stage 'RandomNumber' didn't change, skipping                          core>
Stage 'ComputePower' didn't change, skipping
Running stage 'WriteToFile':
> python3 -c "from src.WriteToFile import WriteToFile; WriteToFile(load=True, name='WriteToFile').run()" 
Updating lock file 'dvc.lock'                                                   

Running stage 'PowerFromFile':
> python3 -c "from src.PowerFromFile import PowerFromFile; PowerFromFile(load=True, name='PowerFromFile').run()" 
Updating lock file 'dvc.lock'                                                   

Running stage 'ComparePowers':
> python3 -c "from src.ComparePowers import ComparePowers; ComparePowers(load=True, name='ComparePowers').run()" 
Updating lock file 'dvc.lock'                                         core>

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


If we now look at our `dvc.yaml` we can see that for our Node dependencies we rely on the `outs/<stage_name>.json` while for the file dependency it is directly connect to the passed file.

In [13]:
from IPython.display import Pretty, display

display(Pretty("dvc.yaml"))

stages:
  RandomNumber:
    cmd: "python3 -c \"from src.RandomNumber import RandomNumber; RandomNumber(load=True,\
      \ name='RandomNumber').run()\" "
    deps:
    - src/RandomNumber.py
    params:
    - RandomNumber
    outs:
    - nodes/RandomNumber/outs.json
  ComputePower:
    cmd: "python3 -c \"from src.ComputePower import ComputePower; ComputePower(load=True,\
      \ name='ComputePower').run()\" "
    deps:
    - nodes/RandomNumber/outs.json
    - src/ComputePower.py
    params:
    - ComputePower
    outs:
    - nodes/ComputePower/outs.json
  WriteToFile:
    cmd: "python3 -c \"from src.WriteToFile import WriteToFile; WriteToFile(load=True,\
      \ name='WriteToFile').run()\" "
    deps:
    - nodes/RandomNumber/outs.json
    - src/WriteToFile.py
    outs:
    - random_number.txt
  PowerFromFile:
    cmd: "python3 -c \"from src.PowerFromFile import PowerFromFile; PowerFromFile(load=True,\
      \ name='PowerFromFile').run()\" "
    deps:
    - random_number.txt
    - src/P

In [14]:
temp_dir.cleanup()